# Using Climate Data Store API to access ERA5 reanalysis  
## Steps (for Windows):  
1. Set up account at: https://cds.climate.copernicus.eu/cdsapp#!/home (button on top right)   
2. From your user profile (click on your name near Logout button on top right) find the API key information -- both the UID and API key.  
  
3. Create a file named .cdsapirc (making a dot file on Windows can be tricky, see instructions links here: https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+Windows). Put three lines of information looking like this into .cdsapirc:  

    url: https://cds.climate.copernicus.eu/api/v2  
    key: insert_your_UID_here : insert_your_API_key_here  
    verify: 0  
      
      
4. Put the .cdsapirc file in your home directory (e.g., C:\Users\carrie.morrill). 
  
5. Install CDS API. E.g., in Anaconda prompt: conda install -c conda-forge cdsapi  
  
6. From the Climate Data Store find the dataset and data you would like (e.g., https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-single-levels?tab=form) and then click button "Show API request" at bottom of page. Copy and paste the API request provided into new notebook cell below.  

7. Note: The API will by default return all grid cell values (global).  To select a latitude-longitude box, use the area selection as shown in the notebook cell below.  For netcdf output, the grid cells are at 0.25 degree intervals.  
  
8. Run API request.  
  
9. Download results from your user account online.  


### Instructions for macOS are at: https://confluence.ecmwf.int/display/CKB/How+to+install+and+use+CDS+API+on+macOS

In [2]:
import cdsapi
c = cdsapi.Client()

In [7]:
c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'format':'netcdf',
        'variable':[
#            '2m_temperature','surface_solar_radiation_downwards','surface_thermal_radiation_downwards'      
#            '10m_u_component_of_wind','10m_v_component_of_wind','2m_dewpoint_temperature'
            'runoff','surface_pressure','total_precipitation'
        ],        
        'year':[
            '2016','2017'
        ],
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'day':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12',
            '13','14','15',
            '16','17','18',
            '19','20','21',
            '22','23','24',
            '25','26','27',
            '28','29','30',
            '31'
        ],
        'area': [68.5, -51.0, 68.5, -51.0], # North, West, South, East. Default: global
        "product_type": "reanalysis",
        'time':[
            '00:00','01:00','02:00',
            '03:00','04:00','05:00',
            '06:00','07:00','08:00',
            '09:00','10:00','11:00',
            '12:00','13:00','14:00',
            '15:00','16:00','17:00',
            '18:00','19:00','20:00',
            '21:00','22:00','23:00'
        ]
    },
    'ro-pr-singlev.nc')

2019-10-11 14:08:17,256 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2019-10-11 14:08:18,135 INFO Request is queued
C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
2019-10-11 14:08:19,294 INFO Request is running
C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS 

C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\carrie.morrill\.conda\envs\carrie-test\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\carrie.morrill\.conda\envs\carrie-

Result(content_length=177016,content_type=application/x-netcdf,location=http://136.156.133.39/cache-compute-0012/cache/data4/adaptor.mars.internal-1570824498.2151284-5958-11-708dee98-d786-426c-af4e-b39d181325a7.nc)

In [139]:
import netCDF4
import numpy as np
import pandas as pd

with Dataset("ts-rad-singlev.nc", mode='r') as fh:  # read in variables from netcdf files
    t2m = fh.variables['t2m'][:].squeeze()
    ssrd = fh.variables['ssrd'][:].squeeze()/(60*60)  # convert J/m2 to W/m2
    strd = fh.variables['strd'][:].squeeze()/(60*60)  # convert J/m2 to W/m2
    time = fh.variables['time'][:]
with Dataset("wind-dew-singlev.nc", mode='r') as fh2:
    u10 = fh2.variables['u10'][:].squeeze()
    v10 = fh2.variables['v10'][:].squeeze()
    d2m = fh2.variables['d2m'][:].squeeze()
with Dataset("ro-ps-pr-singlev.nc", mode='r') as fh3:
    ps = fh3.variables['sp'][:].squeeze()
    pr = fh3.variables['tp'][:].squeeze()
    ro = fh3.variables['ro'][:].squeeze()
wind = np.sqrt(u10*u10+v10*v10)   # calculate wind speed from u and v wind components
ssrd[ssrd < 0] = 0    # change negative shortwave values to zero

hourly = pd.DataFrame(np.column_stack((t2m, d2m, wind, ssrd, strd, ps, pr, ro)))  # combine variables into DataFrame
hourly.columns = ['t2m','hum','wind','sw','lw','ps','precip','runoff']  # label columns
hourly['datetime'] = netCDF4.num2date(time[:], 'hours since 1900-01-01 00:00:00.0', 'gregorian')  # add time column
hourly['Month'] = hourly['datetime'].dt.month  # pull months out of datetime
hourly['Year'] = hourly['datetime'].dt.year  # pull years out of datetime
hourly['Day'] = hourly['datetime'].dt.day  # pull days out of datetime
daily = hourly.groupby(['Year','Month','Day']).mean()  # daily averages
daily['precip'] = daily['precip']*24000  # convert precip from m/hour to mm/day
daily['runoff'] = daily['runoff']*24000  # convert runoff from m/hour to mm/day
daily['hum'] = ((np.exp(17.269*(daily['hum']-273.16)/(daily['hum']-35.86)))/
                (np.exp(17.269*(daily['t2m']-273.16)/(daily['t2m']-35.86)))) *100  # convert dewpt to RH

daily.reset_index(inplace=True)  # rearrange columns for output with lake model ordering
daily.drop(['Month', 'Day'], axis=1, inplace=True)  # drop month and day columns
daily['Day'] = range(1,(len(daily)+1))  # add sequential day column
daily = daily[['Year','Day','t2m','hum','wind','sw','lw','ps','precip','runoff']]  # reorder columns

daily.to_csv('met_input.txt', sep=' ', float_format = "%.4f", header=False, index=False)  # output